# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [ ]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

Please download the Online_retail dataset dataset from [here](https://drive.google.com/file/d/1jD7eQYnfcvbm1ksDbD12U22JtbKItU7E/view?usp=sharing), place it in the data folder.<br>
Load the data:

In [ ]:
data = pd.read_excel('../data/Online_Retail.xlsx')
data


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [ ]:
quantity_revenue=data[['Quantity','Revenue', 'Country', 'InvoiceDate']]
quantity_revenue=quantity_revenue[quantity_revenue.Country != 'United Kingdom']
quantity_revenue=quantity_revenue[quantity_revenue.InvoiceDate >= '2011-04-01']
quantity_revenue=quantity_revenue[quantity_revenue.InvoiceDate <= '2011-04-30']
quantity_revenue.groupby('Country', as_index=False).agg({'Quantity':'sum','Revenue':'sum'})
quantity_revenue.iplot(kind='bar',x='Country',color='purple',title='Total Quantity and Revenue by country for April 2011, excl. UK')
#Paolo: good approach but here you should plot the dataframe produced by the groupby operation. Also this plot shows
# huge numbers they cannot be true given that should be revenue and quantity, so something is wrong. Remember to 
# do sanity checks as you go along. See below 

In [ ]:
#Paolo: plotting the datframe produced by the groupby operation
df=quantity_revenue.groupby('Country', as_index=False).agg({'Quantity':'sum','Revenue':'sum'})
df.iplot(kind='bar',x='Country',title='Total Quantity and Revenue by country for April 2011, excl. UK')


## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [ ]:
quantity_revenue_france=data[['Quantity','Revenue', 'Country', 'InvoiceDate']]
quantity_revenue_france=quantity_revenue_france.loc[quantity_revenue_france.Country == 'France']
quantity_revenue_france=quantity_revenue_france[['Quantity','Revenue','InvoiceDate']]
quantity_revenue_france=quantity_revenue_france[quantity_revenue_france.InvoiceDate >= '2011-01-01']
quantity_revenue_france=quantity_revenue_france[quantity_revenue_france.InvoiceDate <= '2011-05-31']
quantity_revenue_france.groupby('InvoiceDate', as_index=True).agg({'Quantity':'sum','Revenue':'sum'})
quantity_revenue_france


In [ ]:
quantity_revenue_france.iplot(kind='line',x='InvoiceDate',title='Total Quantity and Revenue for France between January 1st 2011 and May 31 2011')
#Paolo: be careful here, good work and good approach but this plot does not make much sense no? What went wrong?
#Paolo: Here you are plotting quantity_revenue_france while you should plot the dataframe produced by the groupby operation, check below

In [ ]:
#Paolo: you see the difference here
df=quantity_revenue_france.groupby('InvoiceDate', as_index=True).agg({'Quantity':'sum','Revenue':'sum'})
df.iplot(kind='line',title='Total Quantity and Revenue for France between January 1st 2011 and May 31 2011')


## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [ ]:
#Paolo: yes good work!
party_bunting=data[['Quantity','UnitPrice','Description','Country']]
party_bunting=party_bunting.loc[party_bunting.Description == 'PARTY BUNTING']
party_bunting=party_bunting[['Quantity','UnitPrice','Country']]
party_bunting=party_bunting.groupby('Country',as_index=False).agg({'Quantity':'mean','UnitPrice':'mean'})
party_bunting.iplot(x='Quantity',y='UnitPrice',categories='Country',xTitle='Average Quantity',yTitle='Average Unit Price', title='Average Quantity & Unit Price for PARTY BUNTING per country')

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [ ]:
quantity_per_invoice=data[['Quantity', 'InvoiceNo', 'Country']]
quantity_per_invoice=quantity_per_invoice[quantity_per_invoice['Country'].isin(['EIRE','Germany','France','Netherlands'])]
quantity_per_invoice=quantity_per_invoice.pivot_table(values='Quantity',columns='Country',index='InvoiceNo',aggfunc='mean')
quantity_per_invoice.iplot(kind='hist',subplots=True, xTitle='Number of Invoices',yTitle='Quantity', title='Distribution of Quantity per Invoice')
#Paolo: yes good work!

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [ ]:
revenue=data[['Revenue','Country','Description']]
revenue=revenue[revenue['Country'].isin(['EIRE','Germany','France','Netherlands'])]
revenue=revenue[revenue['Description'].isin(['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER'])]
revenue=revenue.pivot_table(values='Revenue',columns='Country',index='Description', aggfunc='sum')
revenue.iplot(kind='bar',xTitle='Product', yTitle='Revenue', title='Revenue by country per product')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']
uk

In [ ]:
@interact(Selection=['Year', 'Month'])

def linechart(Selection):
    data = uk.pivot_table(values='Quantity', columns=Selection,
                            index='Day', aggfunc='mean').reset_index()

    data.iplot(kind='line', x='Day', xTitle='Day', 
               yTitle='Quantity', title='Average quantity sold per day in UK ' + Selection.title())
#Paolo: yes, other possibility is to select the month within the year, see below

In [ ]:
#Paolo: possibility with year and month
@interact(Year=uk['Year'].unique(), 
          Month=uk['Month'].unique())

def linechart(Year, Month):
    df = uk[(uk['Year']==Year) & (uk['Month']==Month)]
    grouped = df.groupby('Day').agg('sum')['Quantity']
    grouped.iplot(kind='line', title='UK Sales Quantity by Day')

# Bonus challenges

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.